# Data Modeling with Apache Cassandra

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

## Part I. ETL Pipeline for Pre-Processing the Files

### 1. Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)
#     print(len(file_path_list))

/home/workspace


### 2. Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Part II. Data Modeling in the Apache Cassandra.

The CSV file which needs to be loaded into the cassandra database is titled as <font color=red>event_datafile_new.csv</font>, and located within the Workspace directory.  The `event_datafile_new.csv` contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

### 1. Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # create a session object to establish connection and begin executing queries.
    session = cluster.connect()
except Exception as e:
    print(e)

### 2. Create a Keyspace to Work in

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS dm_cassandra 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

### 3. Set the Keyspace Specified above

In [7]:
try:
    session.set_keyspace('dm_cassandra')
except Exception as e:
    print(e)

As with Apache Cassandra we model the database tables on the queries we want to run. So before I build the data model, I ask the following three questions of the data in order to create our queries.

**Query_1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4**


**Query_2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182**
    

**Query_3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'**




---
### 4. Model the database table based on these queries.  


**Query_1**: Find the `artist`, `song title` and song's `length` in the music app history that was heard during `sessionId = 338`, and `itemInSession = 4`.

In [8]:
# create the table in the cassandra database.
query = "CREATE TABLE IF NOT EXISTS artist_library"
query = query + "(artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [9]:
# insert the data of interest from csv file into the table in cassandra database.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_library (artist, song, length, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [10]:
# verify the data was entered into the table
query = "SELECT * FROM artist_library WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)
    


Faithless Music Matters (Mark Knight Dub) 495.30731201171875


**Now, we can also transform the query result above to a DatFrame for better display.**

In [11]:
query = "SELECT * FROM artist_library WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# set the column names for DataFrame
col_names = ['artist', 'song', 'length']

# construct the dataset fit fot DataFrame
df_vals = []    
for row in rows:
    df_vals.append([row.artist, row.song, row.length])

# transform the query result to a DatFrame
query_df = pd.DataFrame(df_vals, columns=col_names)

# print out the result in DataFrame
query_df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


**Conclusions:** from the query result above, we can see that there is one record which meets `sessionId=338` and `itemsession=4` in the cassandra database.  
The artist name is _Faithless_, the name of the song is _Music Matters (Mark Knight Dub)_, and its duration (length) is _495.307312s_. 

---
**Query_2:** Find only the following: `name` of artist, `song` (_sorted by itemInSession_) and `user` (_first and last name_) for `userid = 10`, `sessionid = 182`.

In [12]:
# create the table based on the second query.
query = "CREATE TABLE IF NOT EXISTS user_library"
query = query + "(artist text, song text, firstName text, lastName text, userId int, sessionId int, PRIMARY KEY (userId, sessionId, song))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [13]:
# insert data from csv into the cassandra database
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_library (artist, song, firstName, lastName, userId, sessionId)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8])))

In [14]:
# verify the data was entered into the table
query = "SELECT * FROM user_library WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)


Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Down To The Bone Keep On Keepin' On Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz


**Now, we can also transform the query result above to a DatFrame for better display.**

In [15]:
query = "SELECT * FROM user_library WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# set the column names for DataFrame    
col_names = ['artist', 'song', 'firstname', 'lastname']
    
# construct the dataset fit fot DataFrame
df_vals = []

for row in rows:
    df_vals.append([row.artist, row.song, row.firstname, row.lastname])

# Transform the query result to a DatFrame
query_df = pd.DataFrame(df_vals, columns=col_names)

# print out the result in DataFrame
query_df

,artist,song,firstname,lastname
0,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
3,Sebastien Tellier,Kilometer,Sylvie,Cruz


**Conclusions:** from the query result above, we can see that there are four records which meets `userId=10` and `sessionId=182` in the cassandra database.  
We can clearly see that result from the DataFrame above. As we we set `sessionId` and `song` as the clustering columns, we can find the values of the song are in __ascending__ order alphabetically.

---
**Query_3**: Find every user `name` (_first and last_) in the music app history who listened to the `song 'All Hands Against His Own'`.

In [16]:
# create the table based on the second query.
query = "CREATE TABLE IF NOT EXISTS log_library"
query = query + "(song text, firstName text, lastName text, userId int, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [17]:
# insert data from csv into the cassandra database
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO log_library (firstName, lastName, song, userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

In [18]:
# verify the data was entered into the table
query = "SELECT * FROM log_library WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


**Next, we can also transform the query result above to a DatFrame for better display.**

In [19]:
# Transfer the query result as a DatFrame
query = "SELECT * FROM log_library WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# set the column names for DataFrame
col_names = ['first_name', 'last_name']

# construct the dataset fit fot DataFrame
df_vals = []
for row in rows:
    df_vals.append([row.firstname, row.lastname])
    
# transfer the dataset to DataFrame
query_df = pd.DataFrame(df_vals, columns=col_names)

# print out the result in DataFrame
query_df    

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


**Conclusions:** from the query result above, we can see that there are three records which meets song name restriction as `song='All Hands Against His Own'` in the cassandra database. We can clearly see that firstname and the lastname of those users who listened to that same song.

### 5. Drop the tables before closing out the sessions

In [20]:
query = "DROP TABLE IF EXISTS artist_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [21]:
query = "DROP TABLE IF EXISTS user_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [22]:
query = "DROP TABLE IF EXISTS log_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### 6. Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()